In this model -- same adam, sigmoids, 12-8-8-1 structure, but batch_size 100, epoch 500. Thus 80 x 500 = 40000 iterations
as Chapter7-6.ipynb. But here it is used to find the change to predictor (prob in last node) based on changes in inputs

In [1]:
#Importing necessary Libraries
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
#Loading Dataset
data = pd.read_csv("Churn_Modelling.csv")

In [3]:
#Generating Dependent Variable Vectors
Y = data.iloc[:,-1].values
X = data.iloc[:,3:13]
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10


In [4]:
#Generating Dependent Variable Vectors
Y = data.iloc[:,-1].values
X = data.iloc[:,3:13]
X['Gender']=X['Gender'].map({'Female':0,'Male':1})
### above is used instead of a more complicated package involving -- from sklearn.preprocessing import LabelEncoder
### converts Female -- 0, Male -- 1, i.e. hot-encoding categorical variables
print (X['Gender'])

0       0
1       0
2       0
3       0
4       0
       ..
9995    1
9996    1
9997    0
9998    1
9999    0
Name: Gender, Length: 10000, dtype: int64


In [5]:
#Encoding Categorical variable Geography
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct =ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[1])],remainder="passthrough")
X = np.array(ct.fit_transform(X))
### Geography is transformed into France -- 1,0,0; Spain -- 0,0,1; Germany -- 0,1,0.
### Moreover -- this encoded vector of ones-zeros is now put in first 3 cols. Credit Score pushed to 4th col.

In [6]:
### convert X to dataframe X1
X1 = pd.DataFrame(X)
X1.head()
### Note there are 12 features including onehotencoder for the Geography feature-- 
### The features are encoded using a one-hot (aka ‘one-of-K’ or ‘dummy’) encoding scheme

,0,1,2,3,4,5,6,7,8,9,10,11
0,1.0,0.0,0.0,619.0,0.0,42.0,2.0,0.00,1.0,1.0,1.0,101348.88
1,0.0,0.0,1.0,608.0,0.0,41.0,1.0,83807.86,1.0,0.0,1.0,112542.58
2,1.0,0.0,0.0,502.0,0.0,42.0,8.0,159660.80,3.0,1.0,0.0,113931.57
3,1.0,0.0,0.0,699.0,0.0,39.0,1.0,0.00,2.0,0.0,0.0,93826.63
4,0.0,0.0,1.0,850.0,0.0,43.0,2.0,125510.82,1.0,1.0,1.0,79084.10


In [7]:
#Splitting dataset into training and testing dataset
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=1)

In [8]:
#Performing Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

We call fit_transform() method on our training data and transform() method on our test data. Each feature in the training
set is scaled to mean 0, variance 1. In sklearn.preprocessing.StandardScaler(), centering and scaling happens independently on each feature. The fit method is calculating the mean and variance of each of the features present in the data. The transform method is transforming all the features using the respective feature's mean and variance that are calculated in the statement
before on X_train.

In [9]:
### This is the very first step while creating NNmodel. Here we are going to create our ann object by using a certain class of Keras 
### named Sequential. As a part of tensorflow 2.0, Keras is now integrated with tensorflow and is now considered as a 
### sub-library of tensorflow. The Sequential class is a part of the models module of Keras library which is a part of the 
### tensorflow library now. 
### It used to be "import tensorflow as tf; from tensorflow import keras; from tensorflow.keras import layers"
### See documentation at https://keras.io/guides/sequential_model/

#Initialising the NN model name -- NNmodel
NNmodel = tf.keras.models.Sequential()

In [10]:
### Creating a network that has 2 hidden layers together with 1 input layer and 1 output layer. 
#Adding First Hidden Layer
NNmodel.add(tf.keras.layers.Dense(units=8,activation="sigmoid"))
### units = 8 refer to 8 neurons in hidden layer 

Above -- first hidden layer is created using the Dense class which is part of the layers module. This class accepts 2 inputs:-
(1) units:- number of neurons that will be present in the respective layer (2) activation:- specify which activation function to be used. This example uses first input as 4. There is no correct answer which is the right number of neurons in the layer -- trial and error. Not too large to be computationally impractical or redundant; not too small to be ineffective.
For the second input, we try the sigmoid or logistic function as an activation function for hidden layers. We can also try “relu”[rectified linear unit].

In [11]:
### Creating 2nd hidden layer 
#Adding Second Hidden Layer -- note this is added sequentially to the first hidden layer
NNmodel.add(tf.keras.layers.Dense(units=8,activation="sigmoid"))
### units = 8 refer to 8 neurons in hidden layer 

In [12]:
### now we create the output layer -- this is added sequentially
#Adding Output Layer
NNmodel.add(tf.keras.layers.Dense(units=1,activation="sigmoid"))
### Only 1 output neuron

For a binary classification problem as above, actual case output is 1 or 0. Hence we require only one neuron to output layer - output could be estimated probability of case actual output = 1. For multiclass classification problem, if the output contains m categories then we need to create m different neurons, one for each category. In the binary output case, the suitable activation function is the sigmoid function. For multiclass classification problem, the activation function is typically softmax. The softmax function predicts a multinomial probability distribution.

In [13]:
### After creating the layers -- require compiling the NNmodel. Compiling allows the computer to run and understand the program 
### without the need of more fundamental steps in the programming. Compiling adds other elements or linking other libraries, and optimization,
### such that after compiling the results are readily computed e.g. in a binary executable program as an output. 
#Compiling NNmodel
NNmodel.compile(optimizer="adam",loss="binary_crossentropy",metrics=['accuracy'])
### Note optimizer here is a more sophisticated version of the Mean Square loss

Compile method above accepts inputs: (1) optimizer:- specifies which optimizer to be used in order to perform stochastic gradient descent (2) error/loss function, e.g., 'binary_crossentropy' here. For multiclass classification, it should be categorical_crossentropy, (3) metrics - the performance metrics to use in order to compute performance. 'accuracy' is one such  performance metric.

In [14]:
X_train.shape

(8000, 12)

In [15]:
#### Last step in creation of NNmodel. NNmodel is trained on the training set here with Tensor-Keras .fit based on Compiler
#Fitting NNmodel
NNmodel.fit(X_train,Y_train,batch_size=100,epochs = 500)

Epoch 1/500
80/80 [==============================] - 0s 606us/step - loss: 0.8684 - accuracy: 0.2030
Epoch 2/500
80/80 [==============================] - 0s 610us/step - loss: 0.6517 - accuracy: 0.6634
Epoch 3/500
80/80 [==============================] - 0s 572us/step - loss: 0.5501 - accuracy: 0.7972
Epoch 4/500
80/80 [==============================] - 0s 560us/step - loss: 0.5086 - accuracy: 0.7972
Epoch 5/500
80/80 [==============================] - 0s 587us/step - loss: 0.4921 - accuracy: 0.7972
Epoch 6/500
80/80 [==============================] - 0s 591us/step - loss: 0.4842 - accuracy: 0.7972
Epoch 7/500
80/80 [==============================] - 0s 557us/step - loss: 0.4788 - accuracy: 0.7972
Epoch 8/500
80/80 [==============================] - 0s 570us/step - loss: 0.4741 - accuracy: 0.7972
Epoch 9/500
80/80 [==============================] - 0s 558us/step - loss: 0.4693 - accuracy: 0.7972
Epoch 10/500
80/80 [==============================] - 0s 562us/step - loss: 0.4646 - accura

80/80 [==============================] - 0s 574us/step - loss: 0.3415 - accuracy: 0.8615
Epoch 161/500
80/80 [==============================] - 0s 557us/step - loss: 0.3413 - accuracy: 0.8606
Epoch 162/500
80/80 [==============================] - 0s 621us/step - loss: 0.3414 - accuracy: 0.8611
Epoch 163/500
80/80 [==============================] - 0s 633us/step - loss: 0.3410 - accuracy: 0.8611
Epoch 164/500
80/80 [==============================] - 0s 570us/step - loss: 0.3408 - accuracy: 0.8621
Epoch 165/500
80/80 [==============================] - 0s 582us/step - loss: 0.3411 - accuracy: 0.8620
Epoch 166/500
80/80 [==============================] - 0s 595us/step - loss: 0.3406 - accuracy: 0.8600
Epoch 167/500
80/80 [==============================] - 0s 570us/step - loss: 0.3403 - accuracy: 0.8618
Epoch 168/500
80/80 [==============================] - 0s 582us/step - loss: 0.3402 - accuracy: 0.8624
Epoch 169/500
80/80 [==============================] - 0s 561us/step - loss: 0.3401 - a

80/80 [==============================] - 0s 557us/step - loss: 0.3274 - accuracy: 0.8627
Epoch 319/500
80/80 [==============================] - 0s 557us/step - loss: 0.3275 - accuracy: 0.8616
Epoch 320/500
80/80 [==============================] - 0s 561us/step - loss: 0.3273 - accuracy: 0.8619
Epoch 321/500
80/80 [==============================] - 0s 544us/step - loss: 0.3275 - accuracy: 0.8619
Epoch 322/500
80/80 [==============================] - 0s 544us/step - loss: 0.3274 - accuracy: 0.8611
Epoch 323/500
80/80 [==============================] - 0s 574us/step - loss: 0.3276 - accuracy: 0.8619
Epoch 324/500
80/80 [==============================] - 0s 544us/step - loss: 0.3274 - accuracy: 0.8608
Epoch 325/500
80/80 [==============================] - 0s 558us/step - loss: 0.3274 - accuracy: 0.8631
Epoch 326/500
80/80 [==============================] - 0s 582us/step - loss: 0.3273 - accuracy: 0.8612
Epoch 327/500
80/80 [==============================] - 0s 557us/step - loss: 0.3270 - a

80/80 [==============================] - 0s 577us/step - loss: 0.3248 - accuracy: 0.8640
Epoch 477/500
80/80 [==============================] - 0s 550us/step - loss: 0.3250 - accuracy: 0.8639
Epoch 478/500
80/80 [==============================] - 0s 560us/step - loss: 0.3249 - accuracy: 0.8639
Epoch 479/500
80/80 [==============================] - 0s 549us/step - loss: 0.3252 - accuracy: 0.8624
Epoch 480/500
80/80 [==============================] - 0s 553us/step - loss: 0.3248 - accuracy: 0.8634
Epoch 481/500
80/80 [==============================] - 0s 570us/step - loss: 0.3252 - accuracy: 0.8643
Epoch 482/500
80/80 [==============================] - 0s 557us/step - loss: 0.3255 - accuracy: 0.8619
Epoch 483/500
80/80 [==============================] - 0s 548us/step - loss: 0.3251 - accuracy: 0.8621
Epoch 484/500
80/80 [==============================] - 0s 557us/step - loss: 0.3250 - accuracy: 0.8629
Epoch 485/500
80/80 [==============================] - 0s 557us/step - loss: 0.3252 - a

In [16]:
NNmodel.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (100, 8)                  104       
                                                                 
 dense_1 (Dense)             (100, 8)                  72        
                                                                 
 dense_2 (Dense)             (100, 1)                  9         
                                                                 
Total params: 185
Trainable params: 185
Non-trainable params: 0
_________________________________________________________________


In [17]:
### Now we use the trained NNmodel to predict output in X_train sample
NNmodel.evaluate(X_train,Y_train)  ### evaluates the loss and accuracy as specified in the Compiler

250/250 [==============================] - 0s 484us/step - loss: 0.3241 - accuracy: 0.8624


[0.324148952960968, 0.862375020980835]

In [18]:
### Now we use the trained weights and biases to try to predict based on new cases 
### Increasing credit score here
tr1=sc.transform([ [1,  0, 0, 400, 1, 40, 3, 60000, 2, 1, 1, 100000],
                   [1,  0, 0, 500, 1, 40, 3, 60000, 2, 1, 1, 100000],
                   [1,  0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 100000],
                   [1,  0, 0, 700, 1, 40, 3, 60000, 2, 1, 1, 100000],
                   [1,  0, 0, 800, 1, 40, 3, 60000, 2, 1, 1, 100000]])
print(tr1)  ### tr.shape is (5,12)
### Predicting result for 5 cases/subjects
print(NNmodel.predict(tr1))
### note in each recompute -- this no. will change slightly because of the random initiation of the weights

[[ 1.00150113e+00 -5.83123916e-01 -5.72731393e-01 -2.58933664e+00
   9.15090654e-01  1.06297721e-01 -7.01742025e-01 -2.63969873e-01
   8.02256955e-01  6.43760170e-01  9.77258519e-01 -2.49134004e-03]
 [ 1.00150113e+00 -5.83123916e-01 -5.72731393e-01 -1.55489968e+00
   9.15090654e-01  1.06297721e-01 -7.01742025e-01 -2.63969873e-01
   8.02256955e-01  6.43760170e-01  9.77258519e-01 -2.49134004e-03]
 [ 1.00150113e+00 -5.83123916e-01 -5.72731393e-01 -5.20462730e-01
   9.15090654e-01  1.06297721e-01 -7.01742025e-01 -2.63969873e-01
   8.02256955e-01  6.43760170e-01  9.77258519e-01 -2.49134004e-03]
 [ 1.00150113e+00 -5.83123916e-01 -5.72731393e-01  5.13974224e-01
   9.15090654e-01  1.06297721e-01 -7.01742025e-01 -2.63969873e-01
   8.02256955e-01  6.43760170e-01  9.77258519e-01 -2.49134004e-03]
 [ 1.00150113e+00 -5.83123916e-01 -5.72731393e-01  1.54841118e+00
   9.15090654e-01  1.06297721e-01 -7.01742025e-01 -2.63969873e-01
   8.02256955e-01  6.43760170e-01  9.77258519e-01 -2.49134004e-03]]
1/1 

In [19]:
### Now we use the trained weights and biases to try to predict based on new cases
### Increasing Tenure here
tr2=sc.transform([ [1,  0, 0, 600, 1, 40, 1, 60000, 2, 1, 1, 100000],
                   [1,  0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 100000],
                   [1,  0, 0, 600, 1, 40, 5, 60000, 2, 1, 1, 100000],
                   [1,  0, 0, 600, 1, 40, 7, 60000, 2, 1, 1, 100000],
                   [1,  0, 0, 600, 1, 40, 9, 60000, 2, 1, 1, 100000]])
print(tr2)  ### tr.shape is (5,12)
### Predicting result for 5 cases/subjects
print(NNmodel.predict(tr2))
### note in each recompute -- this no. will change slightly because of the random initiation of the weights

[[ 1.00150113 -0.58312392 -0.57273139 -0.52046273  0.91509065  0.10629772
  -1.39482057 -0.26396987  0.80225696  0.64376017  0.97725852 -0.00249134]
 [ 1.00150113 -0.58312392 -0.57273139 -0.52046273  0.91509065  0.10629772
  -0.70174202 -0.26396987  0.80225696  0.64376017  0.97725852 -0.00249134]
 [ 1.00150113 -0.58312392 -0.57273139 -0.52046273  0.91509065  0.10629772
  -0.00866348 -0.26396987  0.80225696  0.64376017  0.97725852 -0.00249134]
 [ 1.00150113 -0.58312392 -0.57273139 -0.52046273  0.91509065  0.10629772
   0.68441506 -0.26396987  0.80225696  0.64376017  0.97725852 -0.00249134]
 [ 1.00150113 -0.58312392 -0.57273139 -0.52046273  0.91509065  0.10629772
   1.3774936  -0.26396987  0.80225696  0.64376017  0.97725852 -0.00249134]]
1/1 [==============================] - 0s 0s/step
[[0.02682485]
 [0.0247239 ]
 [0.0220516 ]
 [0.01908722]
 [0.01613007]]


In [20]:
### Now we use the trained weights and biases to try to predict based on new cases
### Increasing Balance here
tr3=sc.transform([ [1,  0, 0, 600, 1, 40, 3, 40000, 2, 1, 1, 100000],
                   [1,  0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 100000],
                   [1,  0, 0, 600, 1, 40, 3, 80000, 2, 1, 1, 100000],
                   [1,  0, 0, 600, 1, 40, 3, 100000, 2, 1, 1, 100000],
                   [1,  0, 0, 600, 1, 40, 3, 120000, 2, 1, 1, 100000]])
print(tr3)  ### tr.shape is (5,12)
### Predicting result for 5 cases/subjects
print(NNmodel.predict(tr3))
### note in each recompute -- this no. will change slightly because of the random initiation of the weights

[[ 1.00150113 -0.58312392 -0.57273139 -0.52046273  0.91509065  0.10629772
  -0.70174202 -0.58464383  0.80225696  0.64376017  0.97725852 -0.00249134]
 [ 1.00150113 -0.58312392 -0.57273139 -0.52046273  0.91509065  0.10629772
  -0.70174202 -0.26396987  0.80225696  0.64376017  0.97725852 -0.00249134]
 [ 1.00150113 -0.58312392 -0.57273139 -0.52046273  0.91509065  0.10629772
  -0.70174202  0.05670408  0.80225696  0.64376017  0.97725852 -0.00249134]
 [ 1.00150113 -0.58312392 -0.57273139 -0.52046273  0.91509065  0.10629772
  -0.70174202  0.37737804  0.80225696  0.64376017  0.97725852 -0.00249134]
 [ 1.00150113 -0.58312392 -0.57273139 -0.52046273  0.91509065  0.10629772
  -0.70174202  0.69805199  0.80225696  0.64376017  0.97725852 -0.00249134]]
1/1 [==============================] - 0s 14ms/step
[[0.02111235]
 [0.0247239 ]
 [0.03696069]
 [0.06679498]
 [0.10933681]]


In [21]:
### Now we use the trained weights and biases to try to predict based on new cases
### Increasing #Products used by customer here
tr4=sc.transform([ [1,  0, 0, 600, 1, 40, 3, 60000, 1, 1, 1, 100000],
                   [1,  0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 100000],
                   [1,  0, 0, 600, 1, 40, 3, 60000, 3, 1, 1, 100000],
                   [1,  0, 0, 600, 1, 40, 3, 60000, 4, 1, 1, 100000]])
print(tr4)  ### tr.shape is (5,12)
### Predicting result for 5 cases/subjects
print(NNmodel.predict(tr4))
### note in each recompute -- this no. will change slightly because of the random initiation of the weights

[[ 1.00150113e+00 -5.83123916e-01 -5.72731393e-01 -5.20462730e-01
   9.15090654e-01  1.06297721e-01 -7.01742025e-01 -2.63969873e-01
  -9.11509571e-01  6.43760170e-01  9.77258519e-01 -2.49134004e-03]
 [ 1.00150113e+00 -5.83123916e-01 -5.72731393e-01 -5.20462730e-01
   9.15090654e-01  1.06297721e-01 -7.01742025e-01 -2.63969873e-01
   8.02256955e-01  6.43760170e-01  9.77258519e-01 -2.49134004e-03]
 [ 1.00150113e+00 -5.83123916e-01 -5.72731393e-01 -5.20462730e-01
   9.15090654e-01  1.06297721e-01 -7.01742025e-01 -2.63969873e-01
   2.51602348e+00  6.43760170e-01  9.77258519e-01 -2.49134004e-03]
 [ 1.00150113e+00 -5.83123916e-01 -5.72731393e-01 -5.20462730e-01
   9.15090654e-01  1.06297721e-01 -7.01742025e-01 -2.63969873e-01
   4.22979001e+00  6.43760170e-01  9.77258519e-01 -2.49134004e-03]]
1/1 [==============================] - 0s 0s/step
[[0.13251618]
 [0.0247239 ]
 [0.90544933]
 [0.9914786 ]]


In [22]:
### Now we use the trained weights and biases to try to predict based on new cases
### Increasing Age here
tr5=sc.transform([ [1,  0, 0, 600, 1, 40, 3, 60000, 2, 0, 1, 100000],
                   [1,  0, 0, 600, 1, 45, 3, 60000, 2, 0, 1, 100000],
                   [1,  0, 0, 600, 1, 50, 3, 60000, 2, 0, 1, 100000],
                   [1,  0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 100000],
                   [1,  0, 0, 600, 1, 45, 3, 60000, 2, 1, 1, 100000],
                   [1,  0, 0, 600, 1, 50, 3, 60000, 2, 1, 1, 100000]])
print(tr5)  ### tr.shape is (5,12)
### Predicting result for 5 cases/subjects
print(NNmodel.predict(tr5))
### note in each recompute -- this no. will change slightly because of the random initiation of the weights

[[ 1.00150113 -0.58312392 -0.57273139 -0.52046273  0.91509065  0.10629772
  -0.70174202 -0.26396987  0.80225696 -1.55337352  0.97725852 -0.00249134]
 [ 1.00150113 -0.58312392 -0.57273139 -0.52046273  0.91509065  0.58393295
  -0.70174202 -0.26396987  0.80225696 -1.55337352  0.97725852 -0.00249134]
 [ 1.00150113 -0.58312392 -0.57273139 -0.52046273  0.91509065  1.06156819
  -0.70174202 -0.26396987  0.80225696 -1.55337352  0.97725852 -0.00249134]
 [ 1.00150113 -0.58312392 -0.57273139 -0.52046273  0.91509065  0.10629772
  -0.70174202 -0.26396987  0.80225696  0.64376017  0.97725852 -0.00249134]
 [ 1.00150113 -0.58312392 -0.57273139 -0.52046273  0.91509065  0.58393295
  -0.70174202 -0.26396987  0.80225696  0.64376017  0.97725852 -0.00249134]
 [ 1.00150113 -0.58312392 -0.57273139 -0.52046273  0.91509065  1.06156819
  -0.70174202 -0.26396987  0.80225696  0.64376017  0.97725852 -0.00249134]]
1/1 [==============================] - 0s 14ms/step
[[0.02646735]
 [0.04499382]
 [0.06801589]
 [0.0247239

In [23]:
### Now we use the trained weights and biases to try to predict based on new cases
### Changing activity: 1=Active, 0=Not active
tr6=sc.transform([ [1,  0, 0, 600, 1, 40, 3, 60000, 2, 1, 0, 100000],
                   [1,  0, 0, 600, 0, 40, 3, 60000, 2, 1, 0, 100000],
                   [0,  1, 0, 600, 1, 40, 3, 60000, 2, 1, 0, 100000],
                   [0,  1, 0, 600, 1, 40, 3, 60000, 2, 1, 0, 100000],
                   [1,  0, 0, 600, 0, 40, 3, 60000, 2, 1, 1, 100000],
                   [1,  0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 100000],
                   [0,  1, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 100000],
                   [0,  1, 0, 600, 0, 40, 3, 60000, 2, 1, 1, 100000]])
print(tr6)  ### tr.shape is (5,12)
### Predicting result for 5 cases/subjects
print(NNmodel.predict(tr6))
### note in each recompute -- this no. will change slightly because of the random initiation of the weights

[[ 1.00150113 -0.58312392 -0.57273139 -0.52046273  0.91509065  0.10629772
  -0.70174202 -0.26396987  0.80225696  0.64376017 -1.02327069 -0.00249134]
 [ 1.00150113 -0.58312392 -0.57273139 -0.52046273 -1.09278791  0.10629772
  -0.70174202 -0.26396987  0.80225696  0.64376017 -1.02327069 -0.00249134]
 [-0.99850112  1.71490137 -0.57273139 -0.52046273  0.91509065  0.10629772
  -0.70174202 -0.26396987  0.80225696  0.64376017 -1.02327069 -0.00249134]
 [-0.99850112  1.71490137 -0.57273139 -0.52046273  0.91509065  0.10629772
  -0.70174202 -0.26396987  0.80225696  0.64376017 -1.02327069 -0.00249134]
 [ 1.00150113 -0.58312392 -0.57273139 -0.52046273 -1.09278791  0.10629772
  -0.70174202 -0.26396987  0.80225696  0.64376017  0.97725852 -0.00249134]
 [ 1.00150113 -0.58312392 -0.57273139 -0.52046273  0.91509065  0.10629772
  -0.70174202 -0.26396987  0.80225696  0.64376017  0.97725852 -0.00249134]
 [-0.99850112  1.71490137 -0.57273139 -0.52046273  0.91509065  0.10629772
  -0.70174202 -0.26396987  0.802

In [24]:
### Now we use the trained weights and biases to try to predict based on new cases
### Increasing estimated salary here
tr7=sc.transform([ [1,  0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000],
                   [1,  0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 60000],
                   [1,  0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 70000],
                   [1,  0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 80000],
                   [1,  0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 90000],
                   [1,  0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 100000]])
print(tr7)  ### tr.shape is (5,12)
### Predicting result for 5 cases/subjects
print(NNmodel.predict(tr7))
### note in each recompute -- this no. will change slightly because of the random initiation of the weights

[[ 1.00150113 -0.58312392 -0.57273139 -0.52046273  0.91509065  0.10629772
  -0.70174202 -0.26396987  0.80225696  0.64376017  0.97725852 -0.87097985]
 [ 1.00150113 -0.58312392 -0.57273139 -0.52046273  0.91509065  0.10629772
  -0.70174202 -0.26396987  0.80225696  0.64376017  0.97725852 -0.69728215]
 [ 1.00150113 -0.58312392 -0.57273139 -0.52046273  0.91509065  0.10629772
  -0.70174202 -0.26396987  0.80225696  0.64376017  0.97725852 -0.52358445]
 [ 1.00150113 -0.58312392 -0.57273139 -0.52046273  0.91509065  0.10629772
  -0.70174202 -0.26396987  0.80225696  0.64376017  0.97725852 -0.34988674]
 [ 1.00150113 -0.58312392 -0.57273139 -0.52046273  0.91509065  0.10629772
  -0.70174202 -0.26396987  0.80225696  0.64376017  0.97725852 -0.17618904]
 [ 1.00150113 -0.58312392 -0.57273139 -0.52046273  0.91509065  0.10629772
  -0.70174202 -0.26396987  0.80225696  0.64376017  0.97725852 -0.00249134]]
1/1 [==============================] - 0s 19ms/step
[[0.02487754]
 [0.02490818]
 [0.02490557]
 [0.0248721